In [5]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
folder_path = 'utkface_aligned_cropped/UTKFace/'

In [54]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [55]:
len(age)

23708

In [56]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [57]:
df.shape

(23708, 3)

In [58]:
df['gender'].unique()

array([0, 1])

In [59]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [60]:
train_df.shape

(20000, 3)

In [61]:
test_df.shape

(3708, 3)

In [62]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [63]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],   
    target_size=(200, 200),
    class_mode='multi_output'
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    directory=folder_path,
    x_col='img',
    y_col=['age', 'gender'],   
    target_size=(200, 200),
    class_mode='multi_output'
)


Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [27]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [64]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [65]:
resnet.trainable=False

output = resnet.layers[-1].output

flatten = Flatten()(output)

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [66]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [67]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})

In [69]:
def multi_output_wrapper(generator):
    for x, y in generator:
        yield x, tuple(y) 


In [ ]:
model.fit(
    multi_output_wrapper(train_generator),
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=multi_output_wrapper(test_generator),
    validation_steps=len(test_generator)
)
